# Retraining after first attempt to prove correct
**Important note:**
In the process of initiating the retraining we found a bug in the environment:
Essentially, the area which now turned out to be buggy has not been trained on originally as it never occurred in the training samples due to a buggy bounds check (mixup between obstacle size `c` and wind speed `w` in `is_in_bounds`

In [1]:
import gym
import time

In [2]:
import pickle
import numpy as np
import polytope as pc

In [3]:
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
import torch
from torch import nn

In [4]:
import acc

In [5]:
torch.manual_seed(42)

In [6]:
env = gym.make('acc-variant-v1')

/home/steuber/anaconda3/envs/nnequiv-tf1/lib/python3.7/site-packages/gym/logger.py:34: UserWarning: WARN: Environment '<class 'acc.ACCEnv2'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [7]:
env.unwrapped.INCLUDE_UNWINNABLE = False

In [8]:
env.seed(42)

[42]

In [9]:
retrain_polytopes = None
with open("acc-2000000-64-64-64-64-polytopes.pickle","rb") as f:
    retrain_polytopes = pickle.load(f)

In [10]:
poly_region = pc.Region(retrain_polytopes)

In [11]:
# Total volume of polytopes?
# Share of instances usually (i.e. without "focus polytopes")

# Volume of state space:
N = 100000
n=2
l_b = np.array([0,-200])
u_b = np.array([100,200])
xs = env.unwrapped.np_random.uniform(low=l_b,high=u_b,size=(N,n))
s = 0
for x in xs:
    if np.sqrt(x[0]*2*env.unwrapped.A)<=x[1] and x[1]<=np.sqrt((env.unwrapped.MAX_VALUE-x[0])*2*env.unwrapped.B) and not (env.unwrapped.is_crash(x) or x[0] > env.unwrapped.MAX_VALUE):
        s+=1
print("s: ",s)
total_vol = np.prod(u_b - l_b) * (s / N)
print("total volume: ", total_vol)

poly_region = pc.Region(retrain_polytopes)
l_b, u_b = poly_region.bounding_box
l_b = l_b.flatten()
u_b = u_b.flatten()
print(l_b,",",u_b)
xs = env.unwrapped.np_random.uniform(low=l_b,high=u_b,size=(N,n))
s = 0
for x in xs:
    if x in poly_region:
        if -np.sqrt(x[0]*2*env.unwrapped.A)<=x[1] and x[1]<=np.sqrt((env.unwrapped.MAX_VALUE-x[0])*2*env.unwrapped.B) and not (env.unwrapped.is_crash(x) or x[0] > env.unwrapped.MAX_VALUE):
            s+=1
poly_vol = np.prod(u_b - l_b) * (s / N)
print("poly volume: ", poly_vol)

# We only have an upper bound for the share, since polytopes may be partially outside the state space of interest
poly_share = poly_vol/total_vol
print("max share: ", poly_share)

s:  19574
total volume:  7829.599999999999
[  -0.      -133.33434] , [84.74737  0.     ]
poly volume:  262.37983345717305
max share:  0.03351126921645717


In [11]:
env.seed(42)
torch.manual_seed(42)

In [14]:
eval_episode_length=1000
training_episode_length=100000

In [15]:
model = PPO.load("ppo_acc_bigger_200000_steps")
model.set_env(env)

env.init_polytopes(1.0,[])
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
mean_reward:2526.17 +/- 2382.14


In [16]:
env.init_polytopes(0.0,retrain_polytopes)
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

Starting separated by  22.797820935679333  meters moving at  -66.94217325840121  m/s.
Starting separated by  4.425616663035938  meters moving at  -27.753048043132083  m/s.
Starting separated by  60.52608137834781  meters moving at  -107.60150968473813  m/s.
Starting separated by  47.76082489165512  meters moving at  -84.75925609205117  m/s.
Starting separated by  0.022113801541307234  meters moving at  -1.3915808646339758  m/s.
Starting separated by  60.223395465414775  meters moving at  -104.98672187627363  m/s.
Starting separated by  1.1674606398086103  meters moving at  -7.330997894428542  m/s.
Starting separated by  3.8946172789040596  meters moving at  -23.60712495043063  m/s.
Starting separated by  0.021351640562011  meters moving at  -1.4129880319628874  m/s.
Starting separated by  58.1975195412723  meters moving at  -98.76786965073552  m/s.
Starting separated by  56.20850542216904  meters moving at  -99.96038110707136  m/s.
Starting separated by  49.37500867059971  meters movin

Starting separated by  57.650023537643435  meters moving at  -102.58823337020753  m/s.
Starting separated by  4.8279040839243265e-05  meters moving at  -0.017218250491579847  m/s.
Starting separated by  62.49135007070616  meters moving at  -111.05674103141652  m/s.
Starting separated by  0.021989801467949266  meters moving at  -1.4177271881786115  m/s.
Starting separated by  54.272413019984356  meters moving at  -96.52450869692608  m/s.
Starting separated by  23.153560794284708  meters moving at  -67.99241433647032  m/s.
Starting separated by  60.8716986305872  meters moving at  -108.28427600961105  m/s.
Starting separated by  48.184136008218715  meters moving at  -90.06494096308197  m/s.
Starting separated by  2.886770694140524  meters moving at  -20.445906106327456  m/s.
Starting separated by  63.077424624025134  meters moving at  -112.25795554281737  m/s.
Starting separated by  0.2368861678760774  meters moving at  -3.7842302263226824  m/s.
Starting separated by  60.45418168292071  

Starting separated by  22.92810798118269  meters moving at  -67.33617394270982  m/s.
Starting separated by  61.83724491141611  meters moving at  -107.54211614125506  m/s.
Starting separated by  23.290040750653187  meters moving at  -66.78852681833077  m/s.
Starting separated by  4.667072959886968  meters moving at  -26.42065018355142  m/s.
Starting separated by  3.8209442673213743  meters moving at  -23.849738539870547  m/s.
Starting separated by  4.754990041557054  meters moving at  -23.795197905482297  m/s.
Starting separated by  23.085010605074686  meters moving at  -67.79560154327885  m/s.
Starting separated by  47.334172931378504  meters moving at  -84.21584619014513  m/s.
Starting separated by  48.97821327392695  meters moving at  -86.44271080322957  m/s.
Starting separated by  0.27668511649592487  meters moving at  -3.797366468725908  m/s.
Starting separated by  61.80152224611612  meters moving at  -107.72375560186498  m/s.
Starting separated by  22.701551310699216  meters movin

Starting separated by  3.287982901676487  meters moving at  -20.49675468697075  m/s.
Starting separated by  52.77112727222186  meters moving at  -93.83337823644578  m/s.
Starting separated by  2.2537779625627112  meters moving at  -11.823002594646468  m/s.
Starting separated by  0.2822306129151125  meters moving at  -4.53171764208823  m/s.
Starting separated by  59.9974074143675  meters moving at  -102.29576183729999  m/s.
Starting separated by  22.827356704124735  meters moving at  -67.04090439865683  m/s.
Starting separated by  49.75235074682244  meters moving at  -94.02751013252013  m/s.
Starting separated by  23.08890536604782  meters moving at  -67.79960483804783  m/s.
Starting separated by  0.03679651244393438  meters moving at  -1.7338118620559992  m/s.
Starting separated by  51.720321560218046  meters moving at  -91.9897040848261  m/s.
Starting separated by  56.26435247619296  meters moving at  -100.1063699608988  m/s.
Starting separated by  52.846257698756865  meters moving at

Starting separated by  0.20161639305504958  meters moving at  -2.7885021577396762  m/s.
Starting separated by  0.32785753823515795  meters moving at  -5.238154980388501  m/s.
Starting separated by  7.100658276077176  meters moving at  -37.018841974824284  m/s.
Starting separated by  22.927049026311135  meters moving at  -67.33325185109885  m/s.
Starting separated by  22.911399588318815  meters moving at  -67.2866592995865  m/s.
Starting separated by  63.5774611408254  meters moving at  -110.83443900347719  m/s.
Starting separated by  0.6165304080265968  meters moving at  -3.8295264058813814  m/s.
Starting separated by  51.903010665899686  meters moving at  -92.33579445104726  m/s.
Starting separated by  0.022733262608796677  meters moving at  -1.4300392560325388  m/s.
Starting separated by  59.68775325916845  meters moving at  -106.03284205819804  m/s.
Starting separated by  22.707434247477433  meters moving at  -66.67333396717927  m/s.
Starting separated by  4.5305737465545155  meters

...

Starting separated by  0.023264549641698017  meters moving at  -1.4482687518527024  m/s.
Starting separated by  46.54918542130926  meters moving at  -82.8243335053221  m/s.
Starting separated by  61.307564108110384  meters moving at  -108.98760850453505  m/s.
Starting separated by  62.4654434086057  meters moving at  -108.33580858702628  m/s.
Starting separated by  48.23956053445983  meters moving at  -85.73667743963732  m/s.
Starting separated by  57.48403333306133  meters moving at  -100.84576903159908  m/s.
Starting separated by  3.446634795988355  meters moving at  -24.6093831934688  m/s.
Starting separated by  23.021378831349754  meters moving at  -67.60877890901769  m/s.
Starting separated by  23.033557574219774  meters moving at  -67.64013351823039  m/s.
Starting separated by  59.40539938502101  meters moving at  -105.69851686311061  m/s.
Starting separated by  22.820970583987673  meters moving at  -67.01224405620131  m/s.
Starting separated by  61.9993762282801  meters moving a

Starting separated by  47.95202595247287  meters moving at  -84.38434024259402  m/s.
Starting separated by  0.021726131919663584  meters moving at  -1.4238446142475272  m/s.
Starting separated by  52.253958669104776  meters moving at  -92.89278352471821  m/s.
Starting separated by  22.960545841192257  meters moving at  -67.42778290568222  m/s.
Starting separated by  60.82131370346905  meters moving at  -108.08468652685225  m/s.
Starting separated by  23.09562973354327  meters moving at  -67.8267628269883  m/s.
Starting separated by  0.03930938864369511  meters moving at  -1.7654333336118073  m/s.
Starting separated by  48.13949195859619  meters moving at  -84.70055122998107  m/s.
Starting separated by  62.45625891994061  meters moving at  -93.03172975989406  m/s.
Starting separated by  62.02557344426969  meters moving at  -110.25439842576338  m/s.
Starting separated by  49.40141964640254  meters moving at  -86.08524198368517  m/s.
Starting separated by  0.030389138416394103  meters mov

Starting separated by  51.605980366114544  meters moving at  -89.7457527465134  m/s.
Starting separated by  0.2733885689204393  meters moving at  -3.7112635427325946  m/s.
Starting separated by  0.03390706959520917  meters moving at  -1.73918590721406  m/s.
Starting separated by  48.83346950532979  meters moving at  -85.96456679914263  m/s.
Starting separated by  0.022962187334599163  meters moving at  -1.425800812251214  m/s.
Starting separated by  22.785116017227256  meters moving at  -66.91709058689696  m/s.
Starting separated by  22.759216200699473  meters moving at  -66.83734058427396  m/s.
Starting separated by  7.039407024345196  meters moving at  -37.46514237694412  m/s.
Starting separated by  0.021658588363876297  meters moving at  -1.4202170329956227  m/s.
Starting separated by  52.71527302340742  meters moving at  -92.96386154993603  m/s.
Starting separated by  2.5637643915061115  meters moving at  -18.446029020941847  m/s.
Starting separated by  60.8992658532417  meters mov

Starting separated by  61.62723497345357  meters moving at  -107.50651584213023  m/s.
Starting separated by  0.02938649263736385  meters moving at  -1.6083085404101753  m/s.
Starting separated by  54.57323443871323  meters moving at  -95.9690496356881  m/s.
Starting separated by  22.73407141835623  meters moving at  -66.76238984286606  m/s.
Starting separated by  0.021888710509495386  meters moving at  -1.4222440463189872  m/s.
Starting separated by  59.181378559039295  meters moving at  -105.22154123062896  m/s.
Starting separated by  60.25678295520295  meters moving at  -105.5699713082214  m/s.
Starting separated by  53.46583931776563  meters moving at  -93.82093425897189  m/s.
Starting separated by  0.021713611544454354  meters moving at  -1.4056411797423778  m/s.
Starting separated by  0.03433169701134301  meters moving at  -1.6465045216385326  m/s.
Starting separated by  22.764418693348325  meters moving at  -66.84714702394734  m/s.
Starting separated by  59.26857303360839  meters

Starting separated by  1.8188810859468  meters moving at  -13.016103288234731  m/s.
Starting separated by  0.021569266786401864  meters moving at  -1.4057205331708904  m/s.
Starting separated by  60.62688484090341  meters moving at  -106.1087733619132  m/s.
Starting separated by  58.46556288119898  meters moving at  -102.70839352204038  m/s.
Starting separated by  56.7703670533066  meters moving at  -100.1712367515237  m/s.
Starting separated by  23.0015166139589  meters moving at  -67.55248497225438  m/s.
Starting separated by  4.214865915035878  meters moving at  -25.078764099414673  m/s.
Starting separated by  49.83658773977659  meters moving at  -86.42209070510802  m/s.
Starting separated by  50.09136800997509  meters moving at  -89.0641055343622  m/s.
Starting separated by  63.92340700620593  meters moving at  -112.86057172865662  m/s.
Starting separated by  0.029160533545103604  meters moving at  -1.6938370163550769  m/s.
Starting separated by  63.94940549530057  meters moving at

In [17]:
results_overall = {}
results_polys = {}

In [18]:
for p in [0.0, 0.1, 0.5, 0.9, 0.95, 1.0]:
    model = PPO.load("ppo_acc_bigger_200000_steps")
    model.set_env(env)
    
    print("p=",p)

    env.init_polytopes(p,retrain_polytopes)
    env.unwrapped.INCLUDE_UNWINNABLE = False
    start_time = time.time()
    model=model.learn(total_timesteps=training_episode_length)
    print("--- %s seconds ---" % (time.time() - start_time))

    model.save("model_backup/acc-2000000-64-64-64-64-100000-"+str(p))

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
Starting separated by  59.048498859416185  meters moving at  -103.38156835213582  m/s.
Starting separated by  61.29728478634516  meters moving at  -109.0553007488468  m/s.
Starting separated by  62.44102905315485  meters moving at  -109.310305833468  m/s.
Starting separated by  2.3673692997113656  meters moving at  -14.64838271154078  m/s.
Starting separated by  0.04184798993982655  meters moving at  -1.7826587086180314  m/s.
Starting separated by  60.015613118152515  meters moving at  -104.49857715965436  m/s.
Starting separated by  50.851135289203874  meters moving at  -88.97654292260945  m/s.
Starting separated by  54.258415335935396  meters moving at  -96.33341868002536  m/s.
Starting separated by  54.99798227070842  meters moving at  -97.84243019249038  m/s.
Starting separated by  60.903615844569146  meters moving at  -106.4660611479094  m/s.
Starting separated by  60.72224410927414  meters moving 

Starting separated by  56.31314494958887  meters moving at  -99.42758881935666  m/s.
Starting separated by  61.86210558299248  meters moving at  -102.42714204141772  m/s.
Starting separated by  51.92245022151726  meters moving at  -90.92439526891039  m/s.
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 241          |
|    ep_rew_mean          | 1.08e+03     |
| time/                   |              |
|    fps                  | 318          |
|    iterations           | 6            |
|    time_elapsed         | 38           |
|    total_timesteps      | 12288        |
| train/                  |              |
|    approx_kl            | 0.0023727797 |
|    clip_fraction        | 0.102        |
|    clip_range           | 0.2          |
|    entropy_loss         | 0.111        |
|    explained_variance   | 0.794        |
|    learning_rate        | 0.0003       |
|    loss                 | 2.8e+03      |
|    n_update

Starting separated by  2.532181319863632  meters moving at  -15.668946410563159  m/s.
Starting separated by  61.01947586788931  meters moving at  -108.45365458349112  m/s.
Starting separated by  51.05553542191625  meters moving at  -90.84305420926066  m/s.
Starting separated by  22.904531195607426  meters moving at  -67.26685465988115  m/s.
Starting separated by  22.814735496265374  meters moving at  -67.00349583326371  m/s.
Starting separated by  5.263641223411611  meters moving at  -31.686393005974747  m/s.
Starting separated by  0.02188342050667857  meters moving at  -1.4027325868510487  m/s.
Starting separated by  52.012392869622374  meters moving at  -92.55328568546565  m/s.
Starting separated by  63.515151557782026  meters moving at  -112.02467732785443  m/s.
Starting separated by  0.23378232935989646  meters moving at  -3.716791439716288  m/s.
Starting separated by  58.35038363198036  meters moving at  -103.72205011351586  m/s.
Starting separated by  0.25233214073594934  meters 

Starting separated by  0.029935587736032283  meters moving at  -1.7182663879344429  m/s.
Starting separated by  23.096578947289125  meters moving at  -67.83293481849147  m/s.
Starting separated by  59.6722843991227  meters moving at  -104.62890065957959  m/s.
Starting separated by  6.248873909607257  meters moving at  -31.87714274159964  m/s.
Starting separated by  0.0415048253119821  meters moving at  -1.7842351985004319  m/s.
Starting separated by  0.022873091052950177  meters moving at  -1.42729455068304  m/s.
Starting separated by  0.44339529670712824  meters moving at  -5.756619532057953  m/s.
Starting separated by  55.60939077881089  meters moving at  -98.9220280937254  m/s.
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 223        |
|    ep_rew_mean          | 883        |
| time/                   |            |
|    fps                  | 301        |
|    iterations           | 16         |
|    time_elapsed      

Starting separated by  53.63544344700891  meters moving at  -95.34782512031882  m/s.
Starting separated by  2.8460668172342127  meters moving at  -19.866030990139393  m/s.
Starting separated by  57.66116407291449  meters moving at  -102.50878225244803  m/s.
Starting separated by  75.81339579812489  meters moving at  -119.78560339828773  m/s.
Starting separated by  53.61406285785656  meters moving at  -93.04442817856757  m/s.
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 254         |
|    ep_rew_mean          | 1.31e+03    |
| time/                   |             |
|    fps                  | 291         |
|    iterations           | 22          |
|    time_elapsed         | 154         |
|    total_timesteps      | 45056       |
| train/                  |             |
|    approx_kl            | 0.026587714 |
|    clip_fraction        | 0.0989      |
|    clip_range           | 0.2         |
|    entropy_loss        

...

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 311          |
|    ep_rew_mean          | 2.21e+03     |
| time/                   |              |
|    fps                  | 249          |
|    iterations           | 8            |
|    time_elapsed         | 65           |
|    total_timesteps      | 16384        |
| train/                  |              |
|    approx_kl            | 0.0010507947 |
|    clip_fraction        | 0.0143       |
|    clip_range           | 0.2          |
|    entropy_loss         | 0.136        |
|    explained_variance   | 0.827        |
|    learning_rate        | 0.0003       |
|    loss                 | 1.66e+03     |
|    n_updates            | 1090         |
|    policy_gradient_loss | -0.000779    |
|    std                  | 0.211        |
|    value_loss           | 8.46e+03     |
------------------------------------------
-----------------------------------------
| rollout/  

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 293          |
|    ep_rew_mean          | 1.97e+03     |
| time/                   |              |
|    fps                  | 226          |
|    iterations           | 17           |
|    time_elapsed         | 153          |
|    total_timesteps      | 34816        |
| train/                  |              |
|    approx_kl            | 0.0044887904 |
|    clip_fraction        | 0.035        |
|    clip_range           | 0.2          |
|    entropy_loss         | 0.208        |
|    explained_variance   | 0.773        |
|    learning_rate        | 0.0003       |
|    loss                 | 6.81e+03     |
|    n_updates            | 1180         |
|    policy_gradient_loss | -0.00492     |
|    std                  | 0.196        |
|    value_loss           | 2.49e+04     |
------------------------------------------
-----------------------------------------
| rollout/  

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 141          |
|    ep_rew_mean          | 278          |
| time/                   |              |
|    fps                  | 218          |
|    iterations           | 26           |
|    time_elapsed         | 243          |
|    total_timesteps      | 53248        |
| train/                  |              |
|    approx_kl            | 0.0062716235 |
|    clip_fraction        | 0.049        |
|    clip_range           | 0.2          |
|    entropy_loss         | 0.242        |
|    explained_variance   | 0.794        |
|    learning_rate        | 0.0003       |
|    loss                 | 1.1e+03      |
|    n_updates            | 1270         |
|    policy_gradient_loss | 0.000204     |
|    std                  | 0.19         |
|    value_loss           | 7.86e+03     |
------------------------------------------
----------------------------------------
| rollout/   

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 128         |
|    ep_rew_mean          | 67.3        |
| time/                   |             |
|    fps                  | 215         |
|    iterations           | 35          |
|    time_elapsed         | 332         |
|    total_timesteps      | 71680       |
| train/                  |             |
|    approx_kl            | 0.002013545 |
|    clip_fraction        | 0.0143      |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.257       |
|    explained_variance   | 0.893       |
|    learning_rate        | 0.0003      |
|    loss                 | 2e+03       |
|    n_updates            | 1360        |
|    policy_gradient_loss | -0.00157    |
|    std                  | 0.187       |
|    value_loss           | 1.4e+04     |
-----------------------------------------
----------------------------------------
| rollout/                |        

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 224        |
|    ep_rew_mean          | 1.24e+03   |
| time/                   |            |
|    fps                  | 213        |
|    iterations           | 44         |
|    time_elapsed         | 421        |
|    total_timesteps      | 90112      |
| train/                  |            |
|    approx_kl            | 0.05108654 |
|    clip_fraction        | 0.394      |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.323      |
|    explained_variance   | 0.573      |
|    learning_rate        | 0.0003     |
|    loss                 | 290        |
|    n_updates            | 1450       |
|    policy_gradient_loss | 0.0733     |
|    std                  | 0.175      |
|    value_loss           | 433        |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_me

In [19]:
# Performance of models on focus polytopes only?
for p in [0.0, 0.1, 0.5, 0.9, 0.95, 1.0]:
    results_overall[p]=[]
    results_polys[p]=[]
    model = PPO.load("model_backup/acc-2000000-64-64-64-64-100000-"+str(p))
    model.set_env(env)
    print("p=",p)
    
    print("Overall:")
    env.init_polytopes(1.0,[])
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_overall[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")
    
    print("Focus Polytopes:")
    env.init_polytopes(0.0,retrain_polytopes)
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_polys[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
Overall:


/home/steuber/anaconda3/envs/nnequiv-tf1/lib/python3.7/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


mean_reward:2929.50 +/- 1987.64
Focus Polytopes:
Starting separated by  48.65412477156396  meters moving at  -86.50789613609018  m/s.
Starting separated by  0.038946361433630644  meters moving at  -1.7254168695723613  m/s.
Starting separated by  22.805102067299472  meters moving at  -66.97139952799529  m/s.
Starting separated by  52.94635616303924  meters moving at  -92.16513946078328  m/s.
Starting separated by  0.02178905543806349  meters moving at  -1.3976428456843655  m/s.
Starting separated by  65.37942779773442  meters moving at  -113.37170948249035  m/s.
Starting separated by  0.2667117933797925  meters moving at  -4.251372036331674  m/s.
Starting separated by  53.66154253362335  meters moving at  -95.4019357778263  m/s.
Starting separated by  73.13763483996577  meters moving at  -115.3844640834528  m/s.
Starting separated by  59.68753175598739  meters moving at  -105.85905091072962  m/s.
Starting separated by  1.049902211007653  meters moving at  -13.719544665004904  m/s.
Start

Starting separated by  7.159495606354351  meters moving at  -35.18716957038302  m/s.
Starting separated by  5.084230174135441  meters moving at  -28.894266035831215  m/s.
Starting separated by  0.0231981791621796  meters moving at  -1.4510031126034213  m/s.
Starting separated by  3.0898174519302906  meters moving at  -19.20467748482559  m/s.
Starting separated by  51.58703086479015  meters moving at  -90.77734745765083  m/s.
Starting separated by  22.789576397796345  meters moving at  -66.91886145904306  m/s.
Starting separated by  62.06097706971679  meters moving at  -110.12250734767049  m/s.
Starting separated by  1.0469903407254346  meters moving at  -13.658453095084372  m/s.
Starting separated by  57.087224794560534  meters moving at  -100.09604690587697  m/s.
Starting separated by  0.0386799040970993  meters moving at  -1.7140183711444181  m/s.
Starting separated by  51.74043201225709  meters moving at  -91.96879273203558  m/s.
Starting separated by  57.65021185347436  meters movi

Starting separated by  53.91273654337906  meters moving at  -84.3757908810841  m/s.
Starting separated by  4.749317356309779  meters moving at  -29.117215217849207  m/s.
Starting separated by  0.021962664159934094  meters moving at  -1.3923363401176765  m/s.
Starting separated by  1.9853579565507  meters moving at  -12.603245125061864  m/s.
Starting separated by  53.49492623393048  meters moving at  -94.40647781821602  m/s.
Starting separated by  6.203420006573828  meters moving at  -32.627581243030946  m/s.
Starting separated by  0.2219077195217818  meters moving at  -3.004568140089739  m/s.
Starting separated by  0.2698769786040104  meters moving at  -3.8353185859070242  m/s.
Starting separated by  62.476041133484934  meters moving at  -108.68221327661581  m/s.
Starting separated by  64.05273209705241  meters moving at  -111.03490770513841  m/s.
Starting separated by  3.3870569049628094  meters moving at  -23.58160355703704  m/s.
Starting separated by  73.68276160956432  meters movin

...

Starting separated by  65.54205325244898  meters moving at  -107.5264989990714  m/s.
Starting separated by  0.31001870933603115  meters moving at  -4.931863172506512  m/s.
Starting separated by  63.22967762210593  meters moving at  -112.38504582893054  m/s.
Starting separated by  2.9446402393585434  meters moving at  -18.333859490767175  m/s.
Starting separated by  58.658747137098835  meters moving at  -102.14500653755107  m/s.
Starting separated by  62.14278597931559  meters moving at  -110.44604584362519  m/s.
Starting separated by  53.12941532888016  meters moving at  -93.09038410509604  m/s.
Starting separated by  0.02618891630983563  meters moving at  -1.5697551781255639  m/s.
Starting separated by  23.04572680489215  meters moving at  -67.67857373679341  m/s.
Starting separated by  51.31244385847144  meters moving at  -89.26335675917565  m/s.
Starting separated by  0.023757188946716343  meters moving at  -1.4641177566446304  m/s.
Starting separated by  51.07310119197472  meters m

Starting separated by  63.67396567233391  meters moving at  -111.6708786755082  m/s.
Starting separated by  59.14455219510597  meters moving at  -104.0960119211367  m/s.
Starting separated by  0.03347618707782571  meters moving at  -1.7276778454393442  m/s.
Starting separated by  56.75929475036884  meters moving at  -99.3556177022291  m/s.
Starting separated by  1.6058487224235272  meters moving at  -10.13839999118752  m/s.
Starting separated by  4.656834565009422  meters moving at  -27.969922210094225  m/s.
Starting separated by  51.1625939789181  meters moving at  -90.12191630488799  m/s.
Starting separated by  50.24128812440934  meters moving at  -89.2632807720945  m/s.
Starting separated by  4.28800845423804e-05  meters moving at  -0.017247044427638775  m/s.
Starting separated by  56.56639318318358  meters moving at  -99.25846200077186  m/s.
Starting separated by  67.63601538584103  meters moving at  -112.26062692100511  m/s.
Starting separated by  22.750047674046474  meters moving

Starting separated by  3.5499507781206727  meters moving at  -20.84482134961695  m/s.
Starting separated by  49.990247340906194  meters moving at  -88.16120001029603  m/s.
Starting separated by  0.029992166321359152  meters moving at  -1.5869730829255926  m/s.
Starting separated by  1.6906033080294842  meters moving at  -10.011468006524932  m/s.
Starting separated by  2.900597737500688  meters moving at  -16.478876653683752  m/s.
Starting separated by  22.868568275285412  meters moving at  -67.15620577290466  m/s.
Starting separated by  0.038137472688572545  meters moving at  -1.7247402895600075  m/s.
Starting separated by  0.029245676227199112  meters moving at  -1.5778419705358417  m/s.
Starting separated by  0.021809863288427662  meters moving at  -1.4172623537256366  m/s.
Starting separated by  22.89868599371536  meters moving at  -67.23831842500594  m/s.
Starting separated by  57.02167951943841  meters moving at  -101.42840435661662  m/s.
Starting separated by  23.070899009614024 

Starting separated by  53.98123615229103  meters moving at  -82.65129164973129  m/s.
Starting separated by  7.01292671890587  meters moving at  -34.67230962718457  m/s.
Starting separated by  53.956998258563814  meters moving at  -95.88616272986648  m/s.
Starting separated by  56.772990188260984  meters moving at  -100.99817004559907  m/s.
Starting separated by  0.02326266320888305  meters moving at  -1.4498178910105677  m/s.
Starting separated by  0.029106673726453737  meters moving at  -1.6694200889160786  m/s.
Starting separated by  52.421544291299725  meters moving at  -93.2082504268495  m/s.
Starting separated by  3.1590480525362374  meters moving at  -20.076840086007152  m/s.
Starting separated by  57.55798108832773  meters moving at  -102.38463187968773  m/s.
Starting separated by  62.06846299796527  meters moving at  -99.24962725377706  m/s.
Starting separated by  3.697086578938567  meters moving at  -22.232931677056467  m/s.
Starting separated by  61.59444621158849  meters mov

Starting separated by  0.02209030266126653  meters moving at  -1.3922064720694591  m/s.
Starting separated by  59.72783376148726  meters moving at  -106.16369264128565  m/s.
Starting separated by  61.546954562078554  meters moving at  -109.47501809361606  m/s.
Starting separated by  0.24241996087054507  meters moving at  -3.561345093367423  m/s.
Starting separated by  73.58035770481024  meters moving at  -116.09860483646462  m/s.
Starting separated by  54.075854378920994  meters moving at  -96.20291691701142  m/s.
Starting separated by  0.28678947016328554  meters moving at  -4.601812125651325  m/s.
Starting separated by  4.234628092059143  meters moving at  -26.105613079417505  m/s.
Starting separated by  53.761221024284545  meters moving at  -95.35674923829365  m/s.
Starting separated by  51.17571294235609  meters moving at  -88.94695718224628  m/s.
Starting separated by  23.416777341267967  meters moving at  -67.19905624424226  m/s.
Starting separated by  2.108347307216426  meters m

In [20]:
print(results_overall)
print(results_polys)

{0.0: [(2929.5005532604455, 1987.642181434125)], 0.1: [(3418.1372081091404, 1656.1341466200433)], 0.5: [(1932.8646950793266, 2531.421546896283)], 0.9: [(1714.7107323160171, 2511.671795638882)], 0.95: [(175.59871589612962, 2224.4011404884886)], 1.0: [(1674.6219416435956, 2785.8237821959356)]}
{0.0: [(4012.506271957636, 42.33747226187223)], 0.1: [(3989.145850322723, 331.1198812155058)], 0.5: [(1604.8040512683392, 2923.258400329725)], 0.9: [(986.1459198029041, 2952.599582192741)], 0.95: [(-1960.620536404848, 174.07861630854438)], 1.0: [(-1903.3717366616727, 737.579486103041)]}


In [22]:
env.seed(1997)
torch.manual_seed(1997)

In [23]:
for p in [0.0, 0.1, 0.5, 0.9, 0.95, 1.0]:
    model = PPO.load("model_backup/acc-2000000-64-64-64-64-100000-"+str(p))
    model.set_env(env)
    
    print("p=",p)

    env.init_polytopes(p,retrain_polytopes)
    start_time = time.time()
    model=model.learn(total_timesteps=training_episode_length)
    print("--- %s seconds ---" % (time.time() - start_time))

    model.save("model_backup/acc-2000000-64-64-64-64-200000-"+str(p))

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
Starting separated by  56.1732895742408  meters moving at  -99.88314500053586  m/s.
Starting separated by  51.09969960519867  meters moving at  -89.69881178408758  m/s.
Starting separated by  1.5447651947614893  meters moving at  -9.0871703578167  m/s.
Starting separated by  56.468366904336555  meters moving at  -100.41059818637223  m/s.
Starting separated by  61.33267588618463  meters moving at  -108.64390962066764  m/s.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 410      |
|    ep_rew_mean     | 3.68e+03 |
| time/              |          |
|    fps             | 505      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------
Starting separated by  53.66847228172836  meters moving at  -98.04369411718383  m/s.
Starting separated by  0.022346370809004548  meters moving at  -1

Starting separated by  58.33924332751992  meters moving at  -103.79347003623995  m/s.
Starting separated by  22.826411409542672  meters moving at  -67.05929164970588  m/s.
Starting separated by  59.80168493258473  meters moving at  -106.39172437186387  m/s.
Starting separated by  62.64954653589666  meters moving at  -110.43974733619564  m/s.
Starting separated by  4.1883302701266425  meters moving at  -25.122259087928587  m/s.
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 410      |
|    ep_rew_mean          | 3.7e+03  |
| time/                   |          |
|    fps                  | 353      |
|    iterations           | 8        |
|    time_elapsed         | 46       |
|    total_timesteps      | 16384    |
| train/                  |          |
|    approx_kl            | 5.343607 |
|    clip_fraction        | 0.0613   |
|    clip_range           | 0.2      |
|    entropy_loss         | 0.959    |
|    explained_variance

Starting separated by  59.88879502245442  meters moving at  -104.55013006659482  m/s.
Starting separated by  49.61878966551485  meters moving at  -88.25623488162493  m/s.
Starting separated by  50.99498463241294  meters moving at  -90.65939700529461  m/s.
Starting separated by  3.3593773127429727  meters moving at  -24.099099141767834  m/s.
Starting separated by  22.990654389564217  meters moving at  -66.82856334139476  m/s.
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 379         |
|    ep_rew_mean          | 3.29e+03    |
| time/                   |             |
|    fps                  | 351         |
|    iterations           | 14          |
|    time_elapsed         | 81          |
|    total_timesteps      | 28672       |
| train/                  |             |
|    approx_kl            | 0.004694607 |
|    clip_fraction        | 0.0968      |
|    clip_range           | 0.2         |
|    entropy_loss        

Starting separated by  64.74730164002433  meters moving at  -112.73582041002469  m/s.
Starting separated by  52.389771780366715  meters moving at  -93.13876176876309  m/s.
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 383        |
|    ep_rew_mean          | 3.35e+03   |
| time/                   |            |
|    fps                  | 346        |
|    iterations           | 20         |
|    time_elapsed         | 118        |
|    total_timesteps      | 40960      |
| train/                  |            |
|    approx_kl            | 0.15334757 |
|    clip_fraction        | 0.0513     |
|    clip_range           | 0.2        |
|    entropy_loss         | 1.13       |
|    explained_variance   | 0.0424     |
|    learning_rate        | 0.0003     |
|    loss                 | 30.8       |
|    n_updates            | 1700       |
|    policy_gradient_loss | 0.0344     |
|    std                  | 0.0772     |
|    val

Starting separated by  47.23011754019959  meters moving at  -83.98105752126756  m/s.
Starting separated by  59.15938684482761  meters moving at  -102.67677869443902  m/s.
Starting separated by  50.790508039288525  meters moving at  -89.5357342394571  m/s.
Starting separated by  23.05733141343397  meters moving at  -67.71141841001834  m/s.
Starting separated by  22.855310733910493  meters moving at  -67.11154196142536  m/s.
Starting separated by  64.02118265115217  meters moving at  -110.91619601493983  m/s.
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 294        |
|    ep_rew_mean          | 2.13e+03   |
| time/                   |            |
|    fps                  | 347        |
|    iterations           | 25         |
|    time_elapsed         | 147        |
|    total_timesteps      | 51200      |
| train/                  |            |
|    approx_kl            | 0.07615604 |
|    clip_fraction        | 0.112   

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 247         |
|    ep_rew_mean          | 1.48e+03    |
| time/                   |             |
|    fps                  | 327         |
|    iterations           | 13          |
|    time_elapsed         | 81          |
|    total_timesteps      | 26624       |
| train/                  |             |
|    approx_kl            | 0.025679864 |
|    clip_fraction        | 0.037       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.384       |
|    explained_variance   | 0.875       |
|    learning_rate        | 0.0003      |
|    loss                 | 187         |
|    n_updates            | 1630        |
|    policy_gradient_loss | 0.000965    |
|    std                  | 0.165       |
|    value_loss           | 2.27e+03    |
-----------------------------------------
------------------------------------------
| rollout/                |      

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 198         |
|    ep_rew_mean          | 815         |
| time/                   |             |
|    fps                  | 337         |
|    iterations           | 22          |
|    time_elapsed         | 133         |
|    total_timesteps      | 45056       |
| train/                  |             |
|    approx_kl            | 0.049352124 |
|    clip_fraction        | 0.105       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.412       |
|    explained_variance   | 0.973       |
|    learning_rate        | 0.0003      |
|    loss                 | 21.4        |
|    n_updates            | 1720        |
|    policy_gradient_loss | 0.0152      |
|    std                  | 0.16        |
|    value_loss           | 461         |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 210         |
|    ep_rew_mean          | 976         |
| time/                   |             |
|    fps                  | 335         |
|    iterations           | 31          |
|    time_elapsed         | 189         |
|    total_timesteps      | 63488       |
| train/                  |             |
|    approx_kl            | 0.009262476 |
|    clip_fraction        | 0.0551      |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.468       |
|    explained_variance   | 0.943       |
|    learning_rate        | 0.0003      |
|    loss                 | 23.5        |
|    n_updates            | 1810        |
|    policy_gradient_loss | 0.0183      |
|    std                  | 0.151       |
|    value_loss           | 1.3e+03     |
-----------------------------------------
----------------------------------------
| rollout/                |        

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 126         |
|    ep_rew_mean          | 154         |
| time/                   |             |
|    fps                  | 338         |
|    iterations           | 40          |
|    time_elapsed         | 241         |
|    total_timesteps      | 81920       |
| train/                  |             |
|    approx_kl            | 0.008011961 |
|    clip_fraction        | 0.0491      |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.477       |
|    explained_variance   | 0.994       |
|    learning_rate        | 0.0003      |
|    loss                 | 205         |
|    n_updates            | 1900        |
|    policy_gradient_loss | 0.00603     |
|    std                  | 0.15        |
|    value_loss           | 845         |
-----------------------------------------
----------------------------------------
| rollout/                |        

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 159        |
|    ep_rew_mean          | 539        |
| time/                   |            |
|    fps                  | 337        |
|    iterations           | 49         |
|    time_elapsed         | 297        |
|    total_timesteps      | 100352     |
| train/                  |            |
|    approx_kl            | 0.16702436 |
|    clip_fraction        | 0.199      |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.521      |
|    explained_variance   | 0.992      |
|    learning_rate        | 0.0003     |
|    loss                 | 227        |
|    n_updates            | 1990       |
|    policy_gradient_loss | 0.0353     |
|    std                  | 0.143      |
|    value_loss           | 564        |
----------------------------------------
--- 299.85693860054016 seconds ---


In [24]:
# Performance of models on focus polytopes only?
for p in [0.0, 0.1, 0.5, 0.9, 0.95, 1.0]:
    model = PPO.load("model_backup/acc-2000000-64-64-64-64-200000-"+str(p))
    model.set_env(env)
    print("p=",p)
    
    print("Overall:")
    env.init_polytopes(1.0,[])
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_overall[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")
    
    print("Focus Polytopes:")
    env.init_polytopes(0.0,retrain_polytopes)
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_polys[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
Overall:
mean_reward:740.00 +/- 2341.57
Focus Polytopes:
Starting separated by  61.51604730638437  meters moving at  -109.44475862434739  m/s.
Starting separated by  0.027073825636633738  meters moving at  -1.5601641538362905  m/s.
Starting separated by  62.891850280530164  meters moving at  -111.7893232526139  m/s.
Starting separated by  0.02315307932782224  meters moving at  -1.434636265034742  m/s.
Starting separated by  62.11013077408407  meters moving at  -110.49738551627726  m/s.
Starting separated by  0.03840141408039912  meters moving at  -1.723304942205573  m/s.
Starting separated by  22.997895531719003  meters moving at  -67.5359837973045  m/s.
Starting separated by  22.89263279675112  meters moving at  -67.22341859050208  m/s.
Starting separated by  0.029497375840280626  meters moving at  -1.7085471139150152  m/s.
Starting separated by  3.058450097066404  meters moving at  -17.187243339377954

Starting separated by  0.021835159223661426  meters moving at  -1.4086088778334944  m/s.
Starting separated by  50.5498606538455  meters moving at  -89.84439768147584  m/s.
Starting separated by  0.11304164899823038  meters moving at  -2.1554824067366534  m/s.
Starting separated by  23.0738177871199  meters moving at  -67.75879933370871  m/s.
Starting separated by  23.040438388708804  meters moving at  -67.66185268547751  m/s.
Starting separated by  62.832753601069804  meters moving at  -111.8172649498504  m/s.
Starting separated by  61.79454153710073  meters moving at  -108.30479504909505  m/s.
Starting separated by  59.393131990419306  meters moving at  -105.64325030412843  m/s.
Starting separated by  0.25639598526820945  meters moving at  -3.4034646965502793  m/s.
Starting separated by  61.251936967458875  meters moving at  -108.96040717758643  m/s.
Starting separated by  4.929401329578562  meters moving at  -29.568890027756645  m/s.
Starting separated by  51.79374781005028  meters 

Starting separated by  48.539602434053535  meters moving at  -84.88106265747135  m/s.
Starting separated by  54.94973551468963  meters moving at  -96.95162928714865  m/s.
Starting separated by  55.08646540881448  meters moving at  -97.99505950711765  m/s.
Starting separated by  5.313616889702395  meters moving at  -32.1711312073631  m/s.
Starting separated by  59.76550674722793  meters moving at  -105.81504876158405  m/s.
Starting separated by  60.07411774111917  meters moving at  -105.387464921876  m/s.
Starting separated by  22.997583802107847  meters moving at  -67.54191523722125  m/s.
Starting separated by  22.799794208321714  meters moving at  -66.94366071911102  m/s.
Starting separated by  49.943453835860765  meters moving at  -87.34919681617009  m/s.
Starting separated by  64.17824396695238  meters moving at  -111.63178356080763  m/s.
Starting separated by  57.06269832108675  meters moving at  -98.92506303217984  m/s.
Starting separated by  53.219405565059056  meters moving at  

Starting separated by  61.499814915236215  meters moving at  -107.55312921207543  m/s.
Starting separated by  4.2310589607514775  meters moving at  -24.713894526328126  m/s.
Starting separated by  0.02272392656793988  meters moving at  -1.446538776243032  m/s.
Starting separated by  58.56533897946511  meters moving at  -104.13153486878966  m/s.
Starting separated by  22.920914946715655  meters moving at  -67.30406441247337  m/s.
Starting separated by  0.3597203410195091  meters moving at  -4.775561049874483  m/s.
Starting separated by  46.54487290376744  meters moving at  -82.73213722639618  m/s.
Starting separated by  22.71832642957387  meters moving at  -66.71279585222108  m/s.
Starting separated by  50.25156633438055  meters moving at  -88.39339342714621  m/s.
Starting separated by  52.84452826554113  meters moving at  -92.43213431811104  m/s.
Starting separated by  49.880699395600885  meters moving at  -88.69816692946  m/s.
Starting separated by  64.14056998797592  meters moving at

Starting separated by  61.37797639955619  meters moving at  -109.09572846022206  m/s.
Starting separated by  56.30403113270039  meters moving at  -100.06050455290425  m/s.
Starting separated by  56.75237948283496  meters moving at  -98.82505562356631  m/s.
Starting separated by  3.9746291923053856  meters moving at  -19.30439621037928  m/s.
Starting separated by  64.46907527035012  meters moving at  -112.6271895795326  m/s.
Starting separated by  62.04776822065248  meters moving at  -110.38268207672708  m/s.
Starting separated by  22.836916676356804  meters moving at  -67.06283290222268  m/s.
Starting separated by  54.46276548456027  meters moving at  -96.91316516134525  m/s.
Starting separated by  57.902603688733784  meters moving at  -103.01991734524667  m/s.
Starting separated by  3.8456549588138627  meters moving at  -27.157104832495822  m/s.
Starting separated by  4.040440272903295  meters moving at  -24.62325542075725  m/s.
Starting separated by  60.10697602165577  meters moving 

...

Starting separated by  53.35402638951722  meters moving at  -94.52290945573895  m/s.
Starting separated by  53.57107466210678  meters moving at  -95.2430722169212  m/s.
Starting separated by  0.14011079550004563  meters moving at  -3.2258586328635257  m/s.
Starting separated by  55.081694330084055  meters moving at  -97.99800883954435  m/s.
Starting separated by  63.153663056827  meters moving at  -112.15653584339631  m/s.
Starting separated by  55.86363626313218  meters moving at  -97.36647484331283  m/s.
Starting separated by  59.48401896704997  meters moving at  -103.11136497795604  m/s.
Starting separated by  0.03873868707837456  meters moving at  -1.747438687233142  m/s.
Starting separated by  22.91647341059774  meters moving at  -67.30068258942401  m/s.
Starting separated by  55.89903243849392  meters moving at  -86.73069519570939  m/s.
Starting separated by  0.032651379727868135  meters moving at  -1.6137289339982828  m/s.
Starting separated by  50.09720696608495  meters moving 

Starting separated by  22.85850359628426  meters moving at  -67.52688125790947  m/s.
Starting separated by  56.32379634661474  meters moving at  -99.69191362374342  m/s.
Starting separated by  0.030044886191662613  meters moving at  -1.6316938667992005  m/s.
Starting separated by  5.553068598068911  meters moving at  -28.914677886152585  m/s.
Starting separated by  50.271747430380344  meters moving at  -88.30504837104411  m/s.
Starting separated by  50.668388692095895  meters moving at  -90.14224292513764  m/s.
Starting separated by  51.76983816866625  meters moving at  -89.9674404661523  m/s.
Starting separated by  49.808217187316366  meters moving at  -87.71720439200617  m/s.
Starting separated by  0.029470376487755842  meters moving at  -1.6895010522743716  m/s.
Starting separated by  22.734600134961237  meters moving at  -66.76866543297827  m/s.
Starting separated by  63.11050692224971  meters moving at  -112.22564495249725  m/s.
Starting separated by  55.78042994954855  meters mov

Starting separated by  0.03848227147065313  meters moving at  -1.7621445657184553  m/s.
Starting separated by  22.905349214975335  meters moving at  -66.6863289788193  m/s.
Starting separated by  46.8590623234658  meters moving at  -83.35257438223975  m/s.
Starting separated by  3.699669490652666  meters moving at  -22.037554327186676  m/s.
Starting separated by  0.2759337970652049  meters moving at  -3.9968973112601187  m/s.
Starting separated by  63.99506640584717  meters moving at  -111.45899703683625  m/s.
Starting separated by  23.005269537208374  meters moving at  -67.55375343502143  m/s.
Starting separated by  0.023483400698576684  meters moving at  -1.4545153490700797  m/s.
Starting separated by  63.314068551703656  meters moving at  -110.65756057867098  m/s.
Starting separated by  22.871380410037514  meters moving at  -67.15558414627372  m/s.
Starting separated by  50.968277087404395  meters moving at  -90.62385900296515  m/s.
Starting separated by  6.2331737973727614  meters 

Starting separated by  23.04521233286357  meters moving at  -67.67274100036943  m/s.
Starting separated by  0.04124480650841643  meters moving at  -1.771147941787727  m/s.
Starting separated by  22.97781283007776  meters moving at  -67.47495825590174  m/s.
Starting separated by  0.03301755087469754  meters moving at  -1.727501587307173  m/s.
Starting separated by  58.19489003452964  meters moving at  -103.45470652949304  m/s.
Starting separated by  59.42773221574916  meters moving at  -104.58008170348361  m/s.
Starting separated by  53.92269246557232  meters moving at  -95.90007201578771  m/s.
Starting separated by  48.30135378678145  meters moving at  -85.88808311973683  m/s.
Starting separated by  0.23856419100907372  meters moving at  -3.751274978279236  m/s.
Starting separated by  61.04902407778075  meters moving at  -106.56840373000739  m/s.
Starting separated by  2.5144565470622333  meters moving at  -15.71925478320405  m/s.
Starting separated by  59.73227621100617  meters moving

Starting separated by  63.71722998794127  meters moving at  -93.17371813935219  m/s.
Starting separated by  51.805549427312435  meters moving at  -92.16055580329117  m/s.
Starting separated by  62.22543408104016  meters moving at  -110.67025718951074  m/s.
Starting separated by  0.4349309023785745  meters moving at  -2.826292519277345  m/s.
Starting separated by  0.022928567184993826  meters moving at  -1.421346903684098  m/s.
Starting separated by  0.022977430218083704  meters moving at  -1.4261477458883534  m/s.
Starting separated by  0.021939905439732076  meters moving at  -1.4229055332594742  m/s.
Starting separated by  0.03402744907201137  meters moving at  -1.6377946126835528  m/s.
Starting separated by  6.017342404641158  meters moving at  -33.22230856657599  m/s.
Starting separated by  55.37805635400823  meters moving at  -96.51051315561747  m/s.
Starting separated by  0.3109729442062012  meters moving at  -4.9561862551212155  m/s.
Starting separated by  23.08044406782632  mete

In [28]:
for p in [0.0, 0.1, 0.5, 0.9, 0.95, 1.0]:
    print("p=",p)
    print(results_overall[p])
    print(results_polys[p])

p= 0.0
[(2929.5005532604455, 1987.642181434125), (740.0034029740095, 2341.5728469501078)]
[(4012.506271957636, 42.33747226187223), (224.28019980418682, 2888.451668489511)]
p= 0.1
[(3418.1372081091404, 1656.1341466200433), (1866.799432562232, 2348.1654507259573)]
[(3989.145850322723, 331.1198812155058), (4027.3148490834237, 38.78762979327285)]
p= 0.5
[(1932.8646950793266, 2531.421546896283), (2298.278150297642, 2471.8655599065737)]
[(1604.8040512683392, 2923.258400329725), (1520.517672857523, 2944.2719326856286)]
p= 0.9
[(1714.7107323160171, 2511.671795638882), (1075.2834744087459, 2571.772665344986)]
[(986.1459198029041, 2952.599582192741), (-128.44361815786363, 2540.549608587723)]
p= 0.95
[(175.59871589612962, 2224.4011404884886), (2903.4489146926403, 2248.3917252352485)]
[(-1960.620536404848, 174.07861630854438), (662.739118689537, 2973.2722718776295)]
p= 1.0
[(1674.6219416435956, 2785.8237821959356), (805.8681440439224, 2708.9763965474235)]
[(-1903.3717366616727, 737.579486103041), 

In [11]:
results_overall={
    0.0:[(2929.5005532604455, 1987.642181434125), (740.0034029740095, 2341.5728469501078)],
    0.1:[(3418.1372081091404, 1656.1341466200433), (1866.799432562232, 2348.1654507259573)],
    0.5:[(1932.8646950793266, 2531.421546896283), (2298.278150297642, 2471.8655599065737)],
    0.9:[(1714.7107323160171, 2511.671795638882), (1075.2834744087459, 2571.772665344986)],
    0.95:[(175.59871589612962, 2224.4011404884886), (2903.4489146926403, 2248.3917252352485)],
    1.0:[(1674.6219416435956, 2785.8237821959356), (805.8681440439224, 2708.9763965474235)]
}
results_polys={
    0.0:[(4012.506271957636, 42.33747226187223), (224.28019980418682, 2888.451668489511)],
    0.1:[(3989.145850322723, 331.1198812155058), (4027.3148490834237, 38.78762979327285)],
    0.5:[(1604.8040512683392, 2923.258400329725), (1520.517672857523, 2944.2719326856286)],
    0.9:[(986.1459198029041, 2952.599582192741), (-128.44361815786363, 2540.549608587723)],
    0.95:[(-1960.620536404848, 174.07861630854438), (662.739118689537, 2973.2722718776295)],
    1.0:[(-1903.3717366616727, 737.579486103041), (-1969.3372723238467, 426.6580722390694)]
}

We evaluate `model_backup/acc-2000000-64-64-64-64-100000-0.1`